<a href="https://colab.research.google.com/github/abhilashhn1993/depression-assessment-on-twitter/blob/Dragon/Final_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import string
import datetime

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.svm import SVC

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [293]:
from google.colab import files
uploaded = files.upload()

Saving LIWC_Topics_AgeGender.xlsx to LIWC_Topics_AgeGender.xlsx


In [294]:
import io
df = pd.read_excel(io.BytesIO(uploaded['LIWC_Topics_AgeGender.xlsx']))

In [ ]:
df.head()

In [ ]:
df.shape

In [297]:
#Data pre-processing
#Leave out Twitter_handle column from the classifier 
df = df.drop(['Twitter_handle'], axis=1)

**###############OPTIONAL(SKIP IF NOT NEEDED)########################**

**Performing One hot encoding to convert categorical features for modeling**

In [298]:
new_cols = ['Age_group','Gender']

In [ ]:
df_proc = pd.get_dummies(df, prefix_sep="_", columns=new_cols)
df_proc.head()

In [300]:
features_for_modeling = list(range(0, df_proc.shape[1]-1))

In [ ]:
#Seperate Train and target from the dataframe
df_proc_train = df_proc.iloc[:,features_for_modeling]
df_proc_train = df_proc.drop(['Target'], axis=1)
df_proc_train.head()

In [ ]:
df_proc_target = df_proc.loc[:,['Target']]
df_proc_target.head()

In [ ]:
df_proc_train.shape

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_proc_train,df_proc_target,test_size=0.3)
sum(Test_Y['Target'] == 1)

In [ ]:
Train_Y.Target.value_counts()

**SKIP to classifier after this line of code**

**###############OPTIONAL CODE ENDS########################**

In [270]:
features_for_modeling = list(range(0, df.shape[1]-1))

In [ ]:
#Seperate Train and target from the dataframe
df_train = df.iloc[:,features_for_modeling]
df_train = df.drop(['Target'], axis=1)
df_train.head()

In [ ]:
df_target = df.loc[:,['Target']]
df_target.head()

In [ ]:
df_train.shape

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_train,df_target,test_size=0.3)
sum(Test_Y['Target'] == 1)

In [ ]:
Train_Y.Target.value_counts()

**CLASSIFIERS**

**Naive Bayes Classifier**

In [306]:
from sklearn.naive_bayes import GaussianNB
model =  GaussianNB()

In [ ]:
nb = model.fit(Train_X, Train_Y)
y_pred_nb = nb.predict(Test_X)

In [ ]:
print("Accuracy:",metrics.accuracy_score(Test_Y, y_pred_nb))

In [ ]:
precision, recall, fscore, support = score(Test_Y, y_pred_nb)
print((recall))

In [ ]:
nb_roc_auc = roc_auc_score(Test_Y, model.fit(Train_X, Train_Y).predict(Test_X))
fpr, tpr, thresholds = roc_curve(Test_Y, model.fit(Train_X, Train_Y).predict_proba(Test_X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % nb_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('NB_ROC_10')
plt.show()

**Logistic Regression**

In [ ]:
logreg = LogisticRegression()
logreg.fit(Train_X, Train_Y)

In [ ]:
y_pred = logreg.predict(Test_X)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(Test_X, Test_Y)))

In [ ]:
precision, recall, fscore, support = score(Test_Y, y_pred)
print((recall))

In [ ]:
logit_roc_auc = roc_auc_score(Test_Y, logreg.predict(Test_X))
fpr, tpr, thresholds = roc_curve(Test_Y, logreg.predict_proba(Test_X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC_10')
plt.show()

**Combination of Models**

In [ ]:
plt.figure()
models = [
          {
              'label': 'Log Reg',
              'model': LogisticRegression(),
          },
          {
              'label': 'N Bayes',
              'model': GaussianNB(),
          }
]

for m in models:
  model = m['model']
  model.fit(Train_X, Train_Y)
  y_pred = model.predict(Test_X)

  fpr, tpr, thresholds = metrics.roc_curve(Test_Y, model.predict_proba(Test_X)[:,1])

  auc = metrics.roc_auc_score(Test_Y, model.predict(Test_X))

  plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))

# Custom settings for the plot
#plt.figure() 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('Model_10ROC')
plt.show()   # Display

**SVM Classifier**

In [ ]:
svclassifier = SVC(C= 1, kernel='linear', degree=3, gamma="auto")
svclassifier.fit(Train_X, Train_Y)

In [ ]:
# Make prediction
y_pred = svclassifier.predict(Test_X)
# Evaluate our model
print("Accuracy :",accuracy_score(Test_Y['Target'],y_pred),"Recall :",recall_score(Test_Y['Target'],y_pred))